<a href="https://colab.research.google.com/github/20134571/20134571.github.io/blob/main/Cats_dogsRev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os, time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
os.listdir("/content/drive/My Drive/Colab Notebooks/Week 12/training_set/training_set")

['dogs', 'cats']

In [24]:
data_dir = "/content/drive/My Drive/Colab Notebooks/Week 12/training_set/training_set"

In [21]:
!ls /content/drive/My Drive/Colab Notebooks/Week 12/training_set/training_set


ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/Colab': No such file or directory
ls: cannot access 'Notebooks/Week': No such file or directory
ls: cannot access '12/training_set/training_set': No such file or directory


In [10]:
!pip install torch torchvision scikit-learn


train_ds = datasets.ImageFolder(root=f"{data_dir}", transform=train_tfms)
test_ds = datasets.ImageFolder(root=f"{data_dir}", transform=test_tfms)


In [ ]:
# ---------------- CONFIG ----------------
data_dir = "/content/drive/My Drive/Colab Notebooks/Week 12/training_set/training_set"
img_size = 128
batch_size = 32
epochs = 8
lr = 1e-3
num_workers = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ---------------- DATA ----------------
train_tfms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])
test_tfms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

#train_ds = datasets.ImageFolder(root=".\data\dogs_cats\training_set", transform=train_tfms)
#train_ds = datasets.ImageFolder(root=f"{data_dir}/training_set", transform=train_tfms)
#test_ds  = datasets.ImageFolder(root=f"{data_dir}/test_set",  transform=test_tfms)
train_ds = datasets.ImageFolder(root=f"{data_dir}", transform=train_tfms)
test_ds  = datasets.ImageFolder(root=f"{data_dir}",  transform=test_tfms)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=num_workers)

class_names = train_ds.classes
num_classes = len(class_names)
print(f"Classes: {class_names}")

# ---------------- MODEL ----------------
class CNN3x3_32Filters(nn.Module):
    def __init__(self, in_channels=3, out_channels=32, num_classes=2):
        super().__init__()
        # Single convolutional layer: 3 input channels (RGB), 32 filters, 3x3 kernel
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))   # global average pooling
        self.fc = nn.Linear(out_channels, num_classes)

    def forward(self, x):
        # input shape: [B, 3, 128, 128]
        x = self.conv(x)     # -> [B, 32, 128, 128]
        x = self.bn(x)
        x = self.act(x)
        x = self.pool(x)     # -> [B, 32, 1, 1]
        x = torch.flatten(x, 1)
        logits = self.fc(x)  # -> [B, 2]
        return logits

model = CNN3x3_32Filters(in_channels=3, out_channels=32, num_classes=num_classes).to(device)
print(model)

# ---------------- LOSS / OPTIMIZER ----------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# ---------------- TRAIN / EVAL ----------------
def train_one_epoch(loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)
    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    preds_all, labels_all = [], []
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)
        total_loss += loss.item() * imgs.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)
        preds_all.extend(logits.argmax(1).cpu().numpy())
        labels_all.extend(labels.cpu().numpy())
    return total_loss / total, correct / total, preds_all, labels_all

# ---------------- TRAINING LOOP ----------------
best_acc = 0.0
for epoch in range(1, epochs + 1):
    tr_loss, tr_acc = train_one_epoch(train_loader)
    te_loss, te_acc, preds, labels = evaluate(test_loader)
    if te_acc > best_acc:
        best_acc = te_acc
        torch.save(model.state_dict(), "best_cnn3x3_32filters.pth")
    print(f"Epoch {epoch}/{epochs} | Train {tr_loss:.4f}, Acc {tr_acc:.3f} | Test {te_loss:.4f}, Acc {te_acc:.3f}")

print(f"\n✅ Done. Best test accuracy: {best_acc:.3f}")
print("\nClassification report:")
print(classification_report(labels, preds, target_names=class_names, digits=3))
print("\nConfusion matrix:")
print(confusion_matrix(labels, preds))

# ----- display the images ---- predicted and true label and the image
# ====== HOLD-OUT VISUAL TEST: 5 cats + 5 dogs from test set ======
import matplotlib.pyplot as plt
from torch.utils.data import Subset

# Helper: get targets robustly across torchvision versions
try:
    test_targets = test_ds.targets  # torchvision>=0.13
except AttributeError:
    test_targets = [s[1] for s in test_ds.samples]

# Find first 5 indices per class (assumes exactly two classes: class_names[0], class_names[1])
cls0_idx = train_ds.class_to_idx[class_names[0]]
cls1_idx = train_ds.class_to_idx[class_names[1]]

cls0_indices, cls1_indices = [], []
for i, t in enumerate(test_targets):
    if t == cls0_idx and len(cls0_indices) < 5:
        cls0_indices.append(i)
    elif t == cls1_idx and len(cls1_indices) < 5:
        cls1_indices.append(i)
    if len(cls0_indices) == 5 and len(cls1_indices) == 5:
        break

assert len(cls0_indices) == 5 and len(cls1_indices) == 5, \
    "Not enough images per class in test set to sample 5 each."

holdout_indices = cls0_indices + cls1_indices
holdout_ds = Subset(test_ds, holdout_indices)
holdout_loader = DataLoader(holdout_ds, batch_size=10, shuffle=False, num_workers=0)

# Inference on the 10 hold-out images
model.eval()
with torch.no_grad():
    images, labels_true = next(iter(holdout_loader))
    images = images.to(device)
    logits = model(images)
    preds = logits.argmax(dim=1).cpu().numpy()
    labels_true = labels_true.numpy()

# Plot: 2 rows x 5 cols
def tensor_to_img(t):
    # t: [C,H,W], no normalization used above, so just permute
    img = t.permute(1, 2, 0).cpu().numpy()
    # clamp just in case any transforms added later
    return img.clip(0, 1)

plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(tensor_to_img(images[i].cpu()))
    pred_name = class_names[preds[i]]
    true_name = class_names[labels_true[i]]
    title = f"Pred: {pred_name}\nTrue: {true_name}"
    plt.title(title, fontsize=10)
    plt.axis("off")
plt.suptitle("Hold-out predictions (5 cats + 5 dogs)", fontsize=14)
plt.tight_layout()
plt.show()

✅ Using device: cuda
Classes: ['cats', 'dogs']
CNN3x3_32Filters(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=32, out_features=2, bias=True)
)


In [ ]:
# Save model Weights
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/Week 12/best_cnn3x3_32filters.pth")
